# Tutorial Advanced Noten: Suche von Intervallfolgen

(version 24_01_2022)

Dieses Tutorial führt in die Suche von Intervallfolgen auf der Grundlage von CAMAT (Computer-Assisted Music Analysis Tool) ein.

Das Durcharbeiten und Nachvollziehen des Tutoriums soll es Ihnen ermöglichen, selbst Intervallfolgen in eigenen Musikbeispielen (Notendateien) zu suchen.

In [3]:
import sys
import os
sys.path.append(os.getcwd().replace(os.path.join('music_xml_parser', 'ipynb'), ''))
import music_xml_parser as mp
from music21 import * 
import csv
from IPython.display import HTML, display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Mit diesen Befehlen wird nicht nur der CAMAT music_xml_parser, 
# sondern darüber hinaus die python-Bibliotheken 'numpy' und 'pandas' für statische Auswertungen, 
# 'music21' sowie 'matplotlib' für grafische Darstellungen geladen. 

environment.set('autoDownload', 'allow')
# Hierdurch wird der Download von xml-Dateien aus dem Internet ermöglicht. 

Zunächst muss eine Notendatei für die Suche ausgewählt, geladen und mit einem Variablennamen (hier: 'xml_files') bezeichnet werden. Wir wählen eine Komposition eines Komponisten, bei dem der Umgang mit Motiven eine besonders große Rolle spielt: den ersten Satz des Streichquartett Op. 18, No. 1, von Ludwig van Beethoven. 

In [4]:
xml_file = 'https://analyse.hfm-weimar.de/database/03/BeLuva_Op18_1-6_1-4_StringQuar_003_00104.xml'

Mit dem folgenden Befehl wird eine Suchanfrage generiert und anschließend eine Ergebnisliste angezeigt. 
Die gesuchte Intervallfolge wird in Halbtonschritten angegeben; die einzelnen Intervallschritte werden durch ein Komma getrennt. 

Ein Beispiel: Suche nach drei Tönen in einer Folge von zwei aufsteigenden großen Sekunden (Ganztönen) 

    interval=[2, 2]
 
Zweites Beispiel: Suche nach drei Tönen, einer großen Terz aufwärts gefolgt von kleiner Sekunde (Halbton) abwärts. 

    interval=[4, -1]

Bei Beethoven wollen wir nach dem Anfangsmotiv F G F E F C suchen. Wie oft und wo genau taucht es im weiteren Verlauf des Satzes auf? In Halbtonschritten ausgedrückt besteht das aus der folgenden Intervallfolge: 

    interval=[2, -2, -1, 1, -5]

In [5]:
results_01 = mp.core.search.simple_interval_search(xml_file,
                                            interval=[2, -2, -1, 1, -5],
                                            return_details=False)
results_01

File at:  ../music_xml_parser/data/xmls_to_parse/hfm_database/BeLuva_Op18_1-6_1-4_StringQuar_003_00104.xml


,Pitch,Occurance
0,B-1,3
1,F,14
2,G-1,1
3,D-1,1


Wie erwartet tritt das Motiv am häufigsten mit dem Anfangston F auf - und zwar 14 Mal. 

ACHTUNG: Dabei steht *-1* für ein b-Vorzeichen, *1* für ein #-Vorzeichen, *0* für ein  Auflösungszeichen, *-2* für ein doppeltes bb usw. Das bedeutet: 
   * B-1 = Bb
   * G-1 = Ges
   * D-1 = Des 

Bei 'return_details=False' werden nur die Häufigkeiten der gesuchten Intervallfolge bezogen auf verschiedene Anfangstöne angezeigt. 

Die Ausgabe kann über den Parameter 'return_details=true' verfeinert werden. Dann werden nicht nur die Häufigkeiten, sondern das genaue Vorkommen angezeigt: in welcher Stimme, mit welchem Anfangston, in welchem Takt und bei welcher Zählzeit (*LocalOnset*). Die letzte Spalte bezieht sich auf die Reihenfolge *aller* Töne im Stück.  

In [6]:
results_02 = mp.core.search.simple_interval_search(xml_file, 
                                            interval=[2, -2, -1, 1, -5],
                                            return_details=True)
results_02

File at:  ../music_xml_parser/data/xmls_to_parse/hfm_database/BeLuva_Op18_1-6_1-4_StringQuar_003_00104.xml


,Pitch,Octave,MIDI,PartName,PartID,Measure,Onset,LocalOnset
0,F,4,65.0,Violin I,1,1,0.0,1.0
1,F,4,65.0,Violin I,1,9,24.0,1.0
2,B-1,5,82.0,Violin I,1,152,453.0,1.0
3,F,5,77.0,Violin I,1,159,474.0,1.0
4,F,5,77.0,Violin I,1,179,534.0,1.0
5,F,4,65.0,Violin I,1,312,933.0,1.0
6,F,4,65.0,Violin II,2,1,0.0,1.0
7,F,4,65.0,Violin II,2,9,24.0,1.0
8,G-1,5,78.0,Violin II,2,155,462.0,1.0
9,D-1,5,73.0,Violin II,2,163,486.0,1.0


Das Motiv beginnt immer am Taktanfang (LocalOnset 1.0) und tritt nie verschoben im Takt auf. Verschoben wäre es z.B., wenn bei LocalOnset 2.0 (auf dem zweiten Schlag) oder 1.5 (auf der dritten Achtel) stehen würde. 

In [7]:
# Mit dem folgenden Befehl lassen sich die Spalten der Tabelle umbenennen. 

# mp.utils.display_table(data=results_02, columns=['Anfangston', 'Oktavlage', 'Midi', 'Stimme', 'PartID', 'Takt', 'Position', 'Onset'])

Ein Blick in die Partitur zeigt, dass oft nur die ersten fünf Töne des Motivs auftreten - gefolgt nicht von einer Quarte, sondern von einem anderen Intervall. Wir wollen daher mit dem nächsten Befehl nach diesem verkürzten Motiv suchen: 

In [12]:
results_03 = mp.core.search.simple_interval_search(xml_file,
                                            interval=[2, -2, -1, 1],
                                            return_details=False)
results_03

File at:  ../music_xml_parser/data/xmls_to_parse/hfm_database/BeLuva_Op18_1-6_1-4_StringQuar_003_00104.xml


,Pitch,Occurance
0,D-1,9
1,G,8
2,E-1,6
3,A-1,6
4,F,44
5,C,19
6,B-1,16
7,G-1,1
8,D,1


Das verkürzte Motiv - ohne den Quartsprung am Schluss bzw. mit einem anderen Folgeintervall - tritt also viel viel häufiger und mit acht unterschiedlichen Anfangstönen auf: Insgesamt kommt es 110 Mal vor! 

### Aufgabe

1. Lassen Sie sich eine detaillierte Liste der Resultate angeben und suchen Sie die einzelnen Motive in der Partitur des Streichquartetts. Wie oft kommt das Motiv mit der gleichen Rhythmisierung wie das Anfangsmotiv vor? Was meinen Sie: welche Resultate mit anderem Rhythmus lassen sich noch als Abwandlungen des Motivs identifizieren? Wie steht es um die Intervallfolgen, die nicht auf dem Taktanfang, sondern auf einer anderen Position im Takt beginnen?

2. Laden Sie die Notendatei einer Komposition Ihrer Wahl, wählen Sie daraus ein prägnantes Motiv - z.B. das Hauptthema zu Beginn - und überprüfen Sie, ob und wie oft das Motiv im weiteren Verlauf des Stückes auftaucht. 